# Test Filters

In [1]:
import os
import time
from Bio.SeqUtils import MeltingTemp as mt
import sys

os.chdir('../' )
cwd = os.getcwd()
dir_output  = os.path.join(cwd, 'oligo_designer_toolsuite/output/annotations')
print(dir_output)
#sys.path.append(cwd)

from oligo_designer_toolsuite.IO import NcbiDB, EnsemblDB, CustomDB
from oligo_designer_toolsuite.oligo_pre_filter._filter_base import MaskedSequences, GCContent, MeltingTemperature
from oligo_designer_toolsuite.oligo_pre_filter._filter_padlock_probes import PadlockArms

/home/francesco/Desktop/Work/oligo-designer-toolsuite/oligo_designer_toolsuite/output/annotations


Create a new Oligo_DB using the filters

In [2]:
Tm_parameters = {
        'check': True,
        'strict': True,
        'c_seq': None,
        'shift': 0,
        'nn_table': getattr(mt,'DNA_NN3'),
        'tmm_table': getattr(mt,'DNA_TMM1'),
        'imm_table': getattr(mt,'DNA_IMM1'),
        'de_table': getattr(mt, 'DNA_DE1'),
        'dnac1': 50, #[nM]
        'dnac2': 0,
        'selfcomp': False,
        'dNTPs': 0,
        'saltcorr': 7,
        'Na': 1.25, #[mM]
        'K': 75, #[mM]
        'Tris': 20, #[mM]
        'Mg': 10, #[mM] 
        }
Tm_correction_parameters = {
        'DMSO': 0,
        'DMSOfactor': 0.75,
        'fmdfactor': 0.65,
        'fmdmethod': 1,
        'GC': None,
        'fmd': 20}

dir_annotation = '/home/francesco/Desktop/Work/NCBI'
annotation = dir_annotation+'/GCF_000001405.40_GRCh38.p14_genomic.gtf'
sequence = dir_annotation+'/GCF_000001405.40_GRCh38.p14_genomic.fna'
genes = ['WASH7P']#, 'DDX11L1', 'TRNT', 'NOC2L', 'PLEKHN1', 'AGRN','UBE2J2', 'DVL1', 'MIB2', 'LOC112268402_1' ]

masked_sequences = MaskedSequences()
GC_content = GCContent(GC_content_min = 40, GC_content_max = 60)
melting_temperature = MeltingTemperature(Tm_min = 52, Tm_max= 67, Tm_parameters = Tm_parameters, Tm_correction_parameters = Tm_correction_parameters)
arms_tm = PadlockArms(min_arm_length=10,max_Tm_dif=2,Tm_min=38,Tm_max=49, Tm_parameters = Tm_parameters, Tm_correction_parameters = Tm_correction_parameters)

filters = [masked_sequences, GC_content, melting_temperature, arms_tm]
custom = CustomDB(probe_length_min = 30, probe_length_max=40, file_annotation=annotation, file_sequence=sequence, filters=filters)

start_time = time.time()
custom.create_oligos_DB(genes = genes, number_batchs = 1, dir_output=dir_output, write_tsv = True)
print(" Creating and filtering the probes: --- %s seconds ---" % (time.time() - start_time))

/home/francesco/Desktop/Work/oligo-designer-toolsuite/oligo_designer_toolsuite/IO/_database.py:58: UserWarning: Species not specified.
  warnings.warn("Species not specified.")
/home/francesco/Desktop/Work/oligo-designer-toolsuite/oligo_designer_toolsuite/IO/_database.py:62: UserWarning: Genome assembly not specified.
  warnings.warn(f"Genome assembly not specified.")
/home/francesco/Desktop/Work/oligo-designer-toolsuite/oligo_designer_toolsuite/IO/_database.py:66: UserWarning: Annotation release not specified.
  warnings.warn(f"Annotation release not specified.")
/home/francesco/Desktop/Work/oligo-designer-toolsuite/oligo_designer_toolsuite/IO/_database.py:70: UserWarning: Annotation source not specified.
  warnings.warn(f"Annotation source not specified.")


 Creating and filtering the probes: --- 8.997250080108643 seconds ---


In [4]:
# analyze the oligos DB
print(custom.oligos_DB.keys())
gene = list(custom.oligos_DB.keys())[0]
seq = list(custom.oligos_DB[gene].keys())[0]
print(seq)
print(custom.oligos_DB[gene][seq])
custom.write_oligos_DB_gtf()
custom.read_oligos_DB_gtf(custom.file_oligos_DB_gtf, custom.file_oligos_DB_fasta)
print(custom.oligos_DB.keys())
gene = list(custom.oligos_DB.keys())[0]
seq = list(custom.oligos_DB[gene].keys())[0]
print(seq)
print(custom.oligos_DB[gene][seq])

dict_keys(['WASH7P'])
WASH7P_104
{'probe_sequence': Seq('AGAGGAGGACGAGGACGACTGGGAATCGTA'), 'transcript_id': ['NR_024540.1'], 'exon_id': ['NR_024540.1_exon11'], 'chromosome': '1', 'start': [14464], 'end': [14494], 'strand': '-', 'length': 30, 'GC_content': 56.67, 'melt_temp': 62.05, 'melt_temp_rm1': 43.24, 'melt_temp_arm2': 44.76, 'dif_melt_temp_arms': 1.52, 'ligation_site': 14}
dict_keys(['WASH7P'])
WASH7P_104
{'probe_sequence': 'AGAGGAGGACGAGGACGACTGGGAATCGTA', 'transcript_id': ['NR_024540.1'], 'exon_id': ['NR_024540.1_exon11'], 'chromosome': '1', 'start': [14464], 'end': [14494], 'strand': '-', 'length': 30, 'GC_content': 56.67, 'melt_temp': 62.05, 'melt_temp_rm1': 43.24, 'melt_temp_arm2': 44.76, 'dif_melt_temp_arms': 1.52, 'ligation_site': 14.0}
